In [1]:
%load_ext autoreload

In [83]:
%autoreload 2
import sys

sys.path.append("../")
from utils.sequence_utils import generate_random_mutant
from utils.model_architectures import Linear, NLNN, CNNa
from models.Noisy_models.Neural_network_models import NN_model
from models.Ground_truth_oracles.RNA_landscape_models import RNA_landscape_constructor
from models.Noisy_models.Ensemble import Ensemble_models
from evaluators.Evaluator import Evaluator
from models.Ground_truth_oracles.TF_binding_landscape_models import *
from utils.sequence_utils import AAS
from explorers.bo_explorer import BO_Explorer

LANDSCAPE_TYPES_PROTEIN = {"RNA": [], "TF": [], "Protein": [1], "GFP": []}
bo_explorer_prod = BO_Explorer(virtual_screen=20, alphabet=AAS)
bo_explorer_prod.debug = False
evaluator_bo = Evaluator(
    bo_explorer_prod,
    landscape_types=LANDSCAPE_TYPES_PROTEIN,
    path="../simulations/Protein/eval_BO/",
)
evaluator_bo.evaluate_for_landscapes(evaluator_bo.efficiency, num_starts=3)


loading landscapes RNA: [], TF: [], Protein: [2], GFP: [], Additive: None
0 TF landscapes loaded.


../models/Ground_truth_oracles/RNA_landscape_models.py:19: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.loaded_landscapes = yaml.load(cfgfile)


1 Protein landscapes loaded.
0 GFP landscapes loaded.
Loading complete.
Running on 2
start seq 2
Evaluating for virtual_screen: 1
Running  NAM {'landscape_id': 2, 'start_id': 2, 'signal_strength': 1}
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
[0.3333333333333333, 0.33333333333333337, 0.33333333333333337]
[0.3333335833561322, 0.33333320832193386, 0.33333320832193386]
[0.33333333333333337, 0.33333333333333337, 0.33333333333333337]
[0.33333335678766723, 0.3333333216061664, 0.3333333216061664]
[0.33333345421513133, 0.33333327289243436, 0.33333327289243436]
[0.33334956909528024, 0.3333252154523599, 0.3333252154523599]
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
[0.3333333333333333, 0.33333333333333337, 0.33333333333333337]
[0.33333342659552373, 0.33333328670223816, 0.33333328670223816]
[0.3333333410542539, 0.33333332947287303, 0.33333332947287303]
Evaluating for virtual_screen: 10
Running  NAM {'landscape_id': 2, 'start_id': 2, 'signal_strength': 1}
[0.333

KeyboardInterrupt: 

In [150]:
from models.Ground_truth_oracles.Protein_landscape_models import Protein_landscape_constructor

landscape_constructor = Protein_landscape_constructor()
landscape_constructor.load_landscapes(landscapes_to_test=[2])

landscape = landscape_constructor.construct_landscape_object(landscape_constructor.loaded_landscapes[0])
gt_oracle = landscape['landscape_oracle']

1 Protein landscapes loaded.


In [151]:
wt_sequence = gt_oracle.pose.sequence()
wt_energy = gt_oracle.get_folding_energy(wt_sequence)
wt_score = gt_oracle.get_fitness(wt_sequence)

print(f'wild-type energy: {wt_energy}, score: {wt_score}\n',
      f'sequence: {wt_sequence}')

wild-type energy: -28.62724472773118, score: 0.5007156806294317
 sequence: MAQASVVANQLIPINTHLTLVMMRSEVVTPVGIPAEDIPRLVSMQVNRAVPLGTTLMPDMVKGYAA


In [147]:
from utils.model_architectures import CNNa
from models.Noisy_models.Neural_network_models import NN_model
from explorers.elitist_explorers import Greedy
import utils.sequence_utils as s_utils

predictor_arch = CNNa(seq_len=len(wt_sequence), alphabet=s_utils.AAS)
predictor = NN_model(gt_oracle, predictor_arch)
predictor.update_model([wt_sequence])

explorer = Greedy(path='../simulations/', virtual_screen=1, alphabet=s_utils.AAS)
explorer.set_model(predictor)

num_rounds = 5
explorer.run(num_rounds)

round: 0, cost: 1, evals: 0, top: 0.12339623134635491
round: 1, cost: 101, evals: 116, top: 0.17550214886925797
round: 2, cost: 201, evals: 236, top: 0.20345357978669543
round: 3, cost: 301, evals: 354, top: 0.2472593161214055
round: 4, cost: 401, evals: 469, top: 0.27939953304496423
MAQASVVANFLIPINTALTLVMAISEVVTPVGIPGEDIPRLVSGEVNHAVPLGTTADPDTVKGTAA


TypeError: '<' not supported between instances of 'float' and 'NoneType'

In [137]:
gt_oracle.get_fitness(wt_sequence)

0.11903376939486365

In [138]:
gt_oracle.get_folding_energy(wt_sequence)

20016.120651648267